# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro
df = pd.read_csv('Gastro_with_MethylomeDetails.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro
(458, 34012)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg12522722,cg11255513,cg09615521,cg12658517,cg02150941,cg22992837,cg02699881,cg05225390,cg01481208,cg08634588
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.892410,0.015756,...,0.909955,0.043985,0.054816,0.904326,0.135138,0.742557,0.029678,0.079017,0.894593,0.902949
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.908813,0.011350,...,0.900398,0.043525,0.043829,0.899376,0.078902,0.754703,0.030224,0.075161,0.909315,0.903716
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.922532,0.013309,...,0.931750,0.051855,0.039594,0.921650,0.074618,0.843395,0.020945,0.058536,0.872011,0.890447
3,200989060105_R03C01,3003,2,1,2,56.7,3,0,0.876056,0.011801,...,0.914384,0.040789,0.048253,0.907792,0.082236,0.833146,0.022764,0.048899,0.889470,0.916905
4,200989060135_R07C01,3004,1,2,0,59.4,3,0,0.813126,0.012203,...,0.898191,0.033763,0.036108,0.891204,0.074322,0.841696,0.020946,0.055184,0.878124,0.902663


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS', 'Gender', 'SCOPA_Gastro_Category','Age','AgeCate'], axis=1)
print(df1.shape)
df1.head(2)

(458, 34005)


,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg12522722,cg11255513,cg09615521,cg12658517,cg02150941,cg22992837,cg02699881,cg05225390,cg01481208,cg08634588
0,2,0.892410,0.015756,0.011101,0.018491,0.020476,0.962482,0.677222,0.029560,0.929821,...,0.909955,0.043985,0.054816,0.904326,0.135138,0.742557,0.029678,0.079017,0.894593,0.902949
1,1,0.908813,0.011350,0.012197,0.022372,0.018128,0.945195,0.730494,0.017911,0.921435,...,0.900398,0.043525,0.043829,0.899376,0.078902,0.754703,0.030224,0.075161,0.909315,0.903716


## RanFor 100 times, CpG only 

In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [5]:
dframe1.to_csv ("Gastro_AllCpG_Accuracy_100RF.csv", index=False)

In [6]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.50,0.100000
1,0.52,0.087178


## RanFor 1000 times. Samples = 50

In [7]:
df2 = df1.drop (['APPRDX'], axis=1)

In [8]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [9]:
print(dframe1.head(3))
dframe1.to_csv ("Gastro_Rand50CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.410  0.120000
1     0.540  0.080000
2     0.515  0.089582


## RanFor 1000 times. Samples = 1

In [10]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(1, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [11]:
print(dframe1.head(3))
dframe1.to_csv ("Gastro_Rand10CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.410  0.120000
1     0.540  0.080000
2     0.515  0.089582
